In [23]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, AveragePooling2D, Conv2D, Flatten
from keras.layers import Lambda
from keras import backend as K # tensorflow

import os
import coremltools
from coremltools.proto import NeuralNetwork_pb2

In [24]:
kears_file = "./KerasMNIST_quantization.h5"
coreml_file_template = './KerasMNIST_quantization_{}.mlmodel'

In [25]:
def build_and_learn_keras_model():
    batch_size = 128
    num_classes = 10
    epochs = 1

    # the data, shuffled and split between train and test sets
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    img_rows = 28
    img_cols = 28

    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    # 入力値の正規化
    x_train /= 255
    x_test /= 255

    # 教師データをクラス分類のデータに変換
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # ネットワーク設計
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same',input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=[3, 3], padding='same', activation='relu'))
    model.add(Flatten())
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(num_classes, activation='softmax'))

    # ネットワークの構成を出力する
    model.summary()

    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model

In [26]:
if not os.path.exists(kears_file):
    keras_model = build_and_learn_keras_model()
    keras_model.save(kears_file)

In [27]:
def quantize(model, name):
    quantizedModel = coremltools.models.neural_network.quantization_utils.quantize_weights(model, 8, name)
    # on macOS10.13 or earlier, a MLModel instance can not be returned.
    if type(quantizedModel) is coremltools.models.MLModel:
        print("ok")
        fileName = coreml_file_template.format(name)
        print(fileName)
        quantizedModel.save(fileName)
    else:
        print("error")

In [29]:
coreml_model = coremltools.converters.keras.convert(kears_file, input_names='image', output_names='digit')

coreml_model.author = u'Yuichi Yoshida'
coreml_model.license = 'MIT'
coreml_model.short_description = u'Quantization'

coreml_model.input_description['image'] = u'入力画像'
coreml_model.output_description['digit'] = u'推定した数字の確率'

coreml_model.save(coreml_file_template.format('none'))

for name in ['linear', 'kmeans', 'linear_lut']:
    quantize(coreml_model, name)

0 : conv2d_1_input, <keras.engine.topology.InputLayer object at 0x13825d510>
1 : conv2d_1, <keras.layers.convolutional.Conv2D object at 0x14be15290>
2 : activation_1, <keras.layers.core.Activation object at 0x150040fd0>
3 : conv2d_2, <keras.layers.convolutional.Conv2D object at 0x14f718d90>
4 : conv2d_2__activation__, <keras.layers.core.Activation object at 0x14b7389d0>
5 : max_pooling2d_1, <keras.layers.pooling.MaxPooling2D object at 0x14b88ff10>
6 : conv2d_3, <keras.layers.convolutional.Conv2D object at 0x13b511e10>
7 : conv2d_3__activation__, <keras.layers.core.Activation object at 0x14b6dbbd0>
8 : flatten_1, <keras.layers.core.Flatten object at 0x13b54ef50>
9 : dense_1, <keras.layers.core.Dense object at 0x13b53ef90>
10 : activation_2, <keras.layers.core.Activation object at 0x13cf589d0>
11 : dense_2, <keras.layers.core.Dense object at 0x14012af90>
12 : dense_2__activation__, <keras.layers.core.Activation object at 0x14b708fd0>
Quantizing using linear quantization
Optimizing Neural